In [1]:
from PIL import Image, ImageFont, ImageDraw
import numpy as np
import imgaug.augmenters as iaa
import os
import cv2
import string
import torch
import torchvision.transforms as transforms
import torchvision
from sklearn.cluster import KMeans

# Define the LeNet model
# Define the LeNet model with features and classifier split
class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # Feature extractor
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(1, 6, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.Conv2d(6, 16, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2)
        )
        # Classifier
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(16 * 4 * 4, 120),
            torch.nn.ReLU(),
            torch.nn.Linear(120, 84),
            torch.nn.ReLU(),
            torch.nn.Linear(84, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 16 * 4 * 4)
        x = self.classifier(x)
        return x


def train_lenet_on_mnist():
    """Train LeNet on the MNIST dataset and save the trained weights."""
    # Define transformations
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Load MNIST dataset
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

    # Initialize the model, loss function, and optimizer
    model = LeNet().to('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    epochs = 10
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(trainloader)}")

    # Save the trained weights
    torch.save(model.state_dict(), 'lenet_mnist_weights.pth')
    print("Training complete. Weights saved as 'lenet_mnist_weights.pth'.")


# Execute the training process
if __name__ == "__main__":
    train_lenet_on_mnist()

# Colab-specific code (if running on Colab)
# Uncomment the following lines if running in Google Colab to download the weights
from google.colab import files
files.download('lenet_mnist_weights.pth')


Using device: cuda
Epoch 1/10, Loss: 0.2650423752261139
Epoch 2/10, Loss: 0.06968034990032566
Epoch 3/10, Loss: 0.04919354182899868
Epoch 4/10, Loss: 0.040808340697493437
Epoch 5/10, Loss: 0.03349337851528008
Epoch 6/10, Loss: 0.02698583355757593
Epoch 7/10, Loss: 0.025715534738022964
Epoch 8/10, Loss: 0.021337849199402032
Epoch 9/10, Loss: 0.018410215855380687
Epoch 10/10, Loss: 0.016442271527574426
Training complete. Weights saved as 'lenet_mnist_weights.pth'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
# Define transformations (must match the ones used during training)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load MNIST test set
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Load trained model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = LeNet().to(device)
model.load_state_dict(torch.load('lenet_mnist_weights.pth', weights_only=True))
model.eval()

# Test set evaluation
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 98.88%
